In [49]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('kaggle-preprocessed.csv')

In [ ]:
df

In [ ]:
df.info()
df.isnull().sum()

In [ ]:
df.groupby('Type_of_file').size().reset_index(name='Count')

In [ ]:
# fillna in Types of files
df['Type_of_file'] = df['Type_of_file'].fillna('CSV')


In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.to_csv('kaggle-prenew.csv', index=False)

In [52]:
df=pd.read_excel("kaggle-prenew.xlsx")

In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your Excel file
df = pd.read_excel("kaggle-prenew.xlsx")

# Convert dataset names to string to avoid errors
df['Dataset_name'] = df['Dataset_name'].astype(str)

# Vectorize using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Dataset_name'])

# Compute similarity
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Create name to index mapping
name_to_index = pd.Series(df.index, index=df['Dataset_name']).drop_duplicates()

# Function to recommend similar datasets
def recommend_similar_datasets(title, num_results=5):
    if title not in name_to_index:
        return f"❌ Dataset '{title}' not found."
    
    idx = name_to_index[title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_results+1]
    dataset_indices = [i[0] for i in sim_scores]
    
    return df[['Dataset_name', 'Dataset_link']].iloc[dataset_indices].reset_index(drop=True)

# Example usage
recommendations = recommend_similar_datasets("Hotel Reservations Dataset", 5)
print(recommendations)


                  Dataset_name  \
0  A hotel's customers dataset   
1         hotel recommendation   
2                 Hotel review   
3       Hotel Booking Dataset    
4               Hotel Booking    

                                        Dataset_link  
0  https://www.kaggle.com/datasets/nantonio/a-hot...  
1  https://www.kaggle.com/datasets/keshavramaiah/...  
2  https://www.kaggle.com/datasets/anu0012/hotel-...  
3  https://www.kaggle.com/datasets/mukuldeshantri...  
4  https://www.kaggle.com/datasets/mojtaba142/hot...  


In [58]:
import pickle

# Save vectorizer
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)


In [59]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset again
df = pd.read_excel("kaggle-prenew.xlsx")
df['Dataset_name'] = df['Dataset_name'].astype(str)

# Load saved TF-IDF
with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

# Transform data
tfidf_matrix = tfidf.transform(df['Dataset_name'])

# Compute similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Save both
with open("df.pkl", "wb") as f:
    pickle.dump(df, f)

with open("cosine_sim.pkl", "wb") as f:
    pickle.dump(cosine_sim, f)

print("✅ df.pkl and cosine_sim.pkl saved successfully!")

✅ df.pkl and cosine_sim.pkl saved successfully!


In [60]:
# Load files
with open("df.pkl", "rb") as f:
    df = pickle.load(f)

with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

with open("cosine_sim.pkl", "rb") as f:
    cosine_sim = pickle.load(f)


In [61]:
import difflib

In [64]:
import pickle
import pandas as pd
import difflib

# Load required files
with open("df.pkl", "rb") as f:
    df = pickle.load(f)

with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf = pickle.load(f)

with open("cosine_sim.pkl", "rb") as f:
    cosine_sim = pickle.load(f)

# Create index mapping
df['Dataset_name'] = df['Dataset_name'].astype(str)
indices = pd.Series(df.index, index=df['Dataset_name']).drop_duplicates()

# Function to print top N similar datasets with similarity scores
def get_top_similarities(title, num=5):
    if title not in indices:
        matches = difflib.get_close_matches(title, indices.index, n=1, cutoff=0.3)
        if not matches:
            print(f"❌ No close match found for: {title}")
            return
        title = matches[0]
        print(f"🔍 Closest match used: {title}")
        
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num+1]
    
    print(f"\n📊 Top {num} similar datasets to: '{title}'\n")
    for i, score in sim_scores:
        print(f"✅ {df.iloc[i]['Dataset_name']} — Similarity Score: {score:.4f}")

# Example usage
get_top_similarities("Netflix")


🔍 Closest match used: Netflix Dataset

📊 Top 5 similar datasets to: 'Netflix Dataset'

✅ Netflix Movies Dataset — Similarity Score: 0.7864
✅ Dataset: NetFlix Shows — Similarity Score: 0.7248
✅ Netflix Shows DataSet — Similarity Score: 0.7248
✅ Most popular on Netflix — Similarity Score: 0.6329
✅ Netflix Dataset Latest 2021 — Similarity Score: 0.6308


In [ ]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity

model = {
    'tfidf': tfidf_vectorizer,
    'cosine_sim': cosine_similarity(tfidf_matrix),
    'df': df
}

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

NameError: name 'tfidf_vectorizer' is not defined

In [6]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load dataset
df = pd.read_excel("kaggle-prenew.xlsx")   # Replace with your file name

# Step 2: Preprocessing (fill missing values)
df['Dataset_name'] = df['Dataset_name'].fillna("")

# Step 3: Create TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Dataset_name'].astype(str))


# Step 4: Prepare model dictionary
model = {
    'tfidf': tfidf_vectorizer,
    'df': df
}

# Step 5: Save the model as model.pkl
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✅ model.pkl created successfully! (TF-IDF + DataFrame only)")


✅ model.pkl created successfully! (TF-IDF + DataFrame only)
